In [ ]:
from openai import OpenAI
from config import OPENAI_API_KEY
import json
import os

client = OpenAI(api_key=OPENAI_API_KEY)

# --- Define functions the AI can call ---
def create_calendar_event(title, date, time, attendee):
    print("\n[Action] Creating Calendar Event:")
    print(f"Title   : {title}")
    print(f"Date    : {date}")
    print(f"Time    : {time}")
    print(f"Attendee: {attendee}")
    print("-" * 40)

def send_email(to, subject, body):
    print("\n[Action] Sending Email:")
    print(f"To     : {to}")
    print(f"Subject: {subject}")
    print(f"Body   : {body}")
    print("-" * 40)

# --- Define tool schemas ---
tools = [
    {
        "type": "function",
        "name": "create_event",
        "description": "Create a calendar event with title, date, time, and attendee.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "date": {"type": "string"},
                "time": {"type": "string"},
                "attendee": {"type": "string"}
            },
            "required": ["title", "date", "time", "attendee"]
        }
    },
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email with a recipient, subject, and body.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string"},
                "subject": {"type": "string"},
                "body": {"type": "string"}
            },
            "required": ["to", "subject", "body"]
        }
    }
]

# --- Load persistent conversation history ---
HISTORY_FILE = "chat_history.json"
if os.path.exists(HISTORY_FILE):
    with open(HISTORY_FILE, "r") as f:
        messages = json.load(f)
else:
    messages = [{"role": "system", "content": "You are a helpful assistant that can create calendar events and send emails."}]

def handle_function_call(function_call):
    """Execute a function returned by the model"""
    name = function_call["name"]
    args = json.loads(function_call["arguments"])
    if name == "create_event":
        create_calendar_event(**args)
    elif name == "send_email":
        send_email(**args)

# --- Chat loop ---
while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        # Save conversation history before exiting
        with open(HISTORY_FILE, "w") as f:
            json.dump(messages, f, indent=2)
        print("Conversation saved. Goodbye!")
        break

    messages.append({"role": "user", "content": user_input})

    # --- Stream response with new Responses API ---
    stream = client.responses.create(
        model="gpt-4.1-mini",
        input=messages,
        tools=tools,
        stream=True
    )
    assistant_message = ""
    full_function_call = ""

    for event in stream:
        # Stream partial text output
        if event.type == "response.output_text.delta":
            print(event.delta, end="", flush=True)
            assistant_message += event.delta

        # Completed event
        elif event.type == "response.completed":
            # Check if there is a function call
            if event.response.output and event.response.output[0].type in ["function_call", "tool_call"]:
                full_function_call = event.response.output[0].arguments
                if full_function_call:
                    function_call = {
                        "name": event.response.output[0].name,
                        "arguments": full_function_call
                    }
                    handle_function_call(function_call)
                    
                    # Store something in assistant message for history
                    if not assistant_message:
                        assistant_message = f"[Function call: {event.response.output[0].name}]"

    # Append assistant message to conversation
    messages.append({"role": "assistant", "content": assistant_message})


# Output Example: 
# [Action] Creating Calendar Event:
# Title   : Team Meeting
# Date    : 2025-11-07
# Time    : 10:00
# Attendee: Alice
# ----------------------------------------
# Conversation saved. Goodbye!


[Action] Creating Calendar Event:
Title   : Team Meeting
Date    : 2025-11-07
Time    : 10:00
Attendee: Alice
----------------------------------------
Conversation saved. Goodbye!
